In [ ]:
import numpy as np 
import pandas as pd 
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as snsa
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv('301.csv')
df.describe(include='all')